In [ ]:
# import sys
# !{sys.executable} -m pip install tensorflow==2.11.0

In [2]:
HOME_DIR = "/home_remote"
HOME = "/home/thi.tra.my.nguyen"

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, Dense, MaxPooling1D, Flatten, Dropout, Activation, Input, Embedding, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import fasttext
import fasttext.util
import os
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import glob

2023-12-10 14:06:38.602074: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 14:06:41.951125: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-10 14:06:41.951250: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-12-10 14:06:41.951267: W tensorfl

In [3]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_memory_growth(gpus[0], True)
    # tf.config.set_logical_device_configuration(
    #     gpus[0],
    #     [tf.config.LogicalDeviceConfiguration(memory_limit=9216)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs;", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 Physical GPUs; 1 Logical GPUs


2023-12-10 14:06:47.488936: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 14:06:48.086487: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10785 MB memory:  -> device: 0, name: Tesla K80, pci bus id: 0000:8d:00.0, compute capability: 3.7


# Preprocess train & test data

Using Wiki pretrained model

In [4]:
ft = fasttext.load_model('/home_remote/fastText/cc.en.300.bin')

In [ ]:
# # Function to get word embeddings for a document
# def get_document_embeddings(document, max_length=100):
#     # Tokenize the document and get the first max_length word vectors
#     tokens = document.split()[:max_length]
    
#     # Apply zero-padding if the document is shorter than max_length
#     if len(tokens) < max_length:
#         padding_count = max_length - len(tokens)
#         tokens.extend(['<PAD>'] * padding_count)

#     # Get word embeddings for each token
#     embeddings = [ft.get_word_vector(token) for token in tokens]

#     return np.array(embeddings)


In [5]:
positives_df_path = os.path.join(HOME_DIR, "positive_df.pkl")
negatives_df_path = os.path.join(HOME_DIR, "negative_df.pkl")

positives = pd.read_pickle(positives_df_path)
negatives = pd.read_pickle(negatives_df_path)

positives['Label'] = 1
negatives['Label'] = 0

#concatenate the two dataframes
df = pd.concat([positives, negatives], ignore_index=True)
df['text'] = df['Text'] + df['Title']

In [6]:
#shuffle the dataframe and train-test split
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
X = df['text']
y = df['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#convert y_train and y_test to numpy arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()


In [8]:
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [9]:
def get_documents_matrix(documents, max_words=100, embedding_dim=300):
    document_matrices = []

    for document in documents:
        # Split document into words
        words = document.split()[:max_words]

        # Get word embeddings using FastText
        embeddings = [ft.get_word_vector(word) for word in words]

        document_matrices.append(embeddings)

    # Pad each sequence of embeddings to a common length
    padded_document_matrices = pad_sequences(document_matrices, maxlen=max_words, dtype='float32', padding='post', truncating='post')

    return padded_document_matrices


In [11]:
#X_train_emdedded = get_documents_matrix(X_train)
X_test_emdedded = get_documents_matrix(X_test)

In [ ]:
#convert the whole dataset to embedding
X_embedded = get_documents_matrix(X)
#convert to array and reshape y to 2D
y_embedded= np.array(y).reshape(-1,1)


## Save train and test data to disk

In [ ]:
def save_data_to_disk(np_array, file_name_prefix, file_type="npy"):
    """
    Split the np_array into 10 chunks and save them to csv files
    """
    chunks = np.array_split(np_array, 10)
    for i in range(len(chunks)):
        filename = os.path.join(HOME_DIR, "master_thesis/data", "{}_{}.{}".format(file_name_prefix, i + 1, file_type))
        if file_type == "npy":
            np.save(filename, chunks[i])
        elif file_type == "csv":
            np.savetxt(filename, chunks[i])
        else:
            raise ValueError("file_type must be either npy or csv")

In [ ]:
save_data_to_disk(X_train_emdedded, "X_train_emdedded", file_type="npy")

In [ ]:
save_data_to_disk(y_train, "y_train", file_type="npy")

In [ ]:
save_data_to_disk(X_embedded, "X_embedded", file_type="npy")
save_data_to_disk(y_embedded, "y_embedded", file_type="npy")

# Train model

## Create `tf.data.Dataset` object to store train data

In [ ]:
def generator_function(X_train_path, y_train_path):
    X_train_file_list = glob.glob(X_train_path)
    X_train_file_list.sort()
    y_train_file_list = glob.glob(y_train_path)
    y_train_file_list.sort()

    for npy_file_x, npy_file_y in zip(X_train_file_list, y_train_file_list):
        data_x = np.load(npy_file_x, mmap_mode='r')
        data_y = np.load(npy_file_y, mmap_mode='r')

        batch_size = 32
        for i in range(0, len(data_x), batch_size):
            yield data_x[i:i + batch_size], data_y[i:i + batch_size]

X_train_filename_pattern = os.path.join(HOME_DIR, "master_thesis/data/X_embedded*.npy")
X_train_shape = (None, 100, 300)
y_train_filename_pattern = os.path.join(HOME_DIR, "master_thesis/data/y_embedded*.npy")
y_train_shape = (None, 1)
dataset = tf.data.Dataset.from_generator(
    generator_function, 
    args=(X_train_filename_pattern, y_train_filename_pattern), 
    output_signature=(
        tf.TensorSpec(shape=X_train_shape, dtype=tf.float32),
        tf.TensorSpec(shape=y_train_shape, dtype=tf.int32)
    )
)

## Construct Sequential models

In [ ]:
# Custom CReLU activation function
def crelu(x):
    pos = K.relu(x)
    neg = K.relu(-x)
    return K.concatenate([pos, neg], axis=-1)

In [ ]:
# Word embedding dimensionality
word_vector_dimensions = 300

# Model
model2 = Sequential()

# Convolutional Layer
model2.add(Conv1D(filters=100, kernel_size=2, input_shape=(100, 300)))
model2.add(Activation(crelu))
model2.add(Activation(lambda x: x * -1))  # Negated activation for CReLU

# 1-Max Pooling Layer
model2.add(MaxPooling1D(pool_size=99))

# Fully Connected Layer
model2.add(Flatten())
model2.add(Dense(200))

# Dropout Layer
model2.add(Dropout(0.4))

# 3 Fully Connected Layers
model2.add(Dense(100, activation=crelu))
model2.add(Dense(50, activation=crelu))
model2.add(Dense(25, activation=crelu))

# Final Layer
model2.add(Dense(1, activation='sigmoid'))


# Display the model summary
model2.summary()

model2.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=1e-4), metrics=['accuracy'])


## Train model with `tf.data.DataSet`

In [ ]:
model2.fit(dataset, epochs=30, batch_size=10000)

# Save model

In [ ]:
#save model
model2.save(os.path.join(HOME_DIR, "master_thesis/cnn_model2.h5"))

In [ ]:
txt = [X_test[48987]]
vc = get_documents_matrix(txt)

In [ ]:
#tf.keras.models.save_model(my_model, './saved_models/my_tf_model')
model2.save(os.path.join(HOME_DIR, "master_thesis/cnn_model2"))

In [14]:
my_tf_saved_model = tf.keras.models.load_model(os.path.join(HOME_DIR, "master_thesis/cnn_model2"))
my_tf_saved_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_5 (Conv1D)           (None, 99, 100)           60100     
                                                                 
 activation_10 (Activation)  (None, 99, 200)           0         
                                                                 
 activation_11 (Activation)  (None, 99, 200)           0         
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 1, 200)           0         
 1D)                                                             
                                                                 
 flatten_5 (Flatten)         (None, 200)               0         
                                                                 
 dense_26 (Dense)            (None, 200)               40200     
                                                      

In [16]:
re = my_tf_saved_model.predict(X_test_emdedded)

2023-12-10 14:11:18.426907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8200
2023-12-10 14:11:18.731953: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.92GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


1857/1857 [==============================] - 13s 6ms/step


2023-12-10 14:11:30.703249: W tensorflow/tsl/framework/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.29GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [22]:
#get 98 percentiles of re
np.percentile(re, 98)

0.8248120951652527